# Big Data Project
## Outcome: Divergence

In this notebook we will outcome 'Divergence'

'Divergence' will be used to detect matches with a big difference between bet houses quotas.


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DataType, DecimalType
from pyspark.sql.functions import udf
import datetime
import pyspark

spark = SparkSession \
    .builder \
    .appName("Bets Exploring") \
    .getOrCreate()
    
sc = spark.sparkContext

sc

<SparkContext master=local[*] appName=Bets Exploring>

In [2]:
pathori = "../Data/Raw/Main/"
pathtemp = "../Data/Interim/"
pathdest = "../Data/Processed/"
filetemp = "main_competitions.csv"
filedest = "main_competitions_recent.csv"
bethouses = ['B365', 'BS', 'BW', 'GB', 'IW', 'LB', 'SB', 'SJ', 'VC', 'WH']
quotatypes = ['H','D','A']

In [3]:
# Open dataset
df = spark.read.csv(path = pathdest + filedest, header = True, inferSchema = True)
#df = df.withColumn("Date", df["Date"].cast("date"))

In [4]:
print ("Size:   ", df.count())
print ("Columns:", df.columns)
df.printSchema()

Size:    11374
Columns: ['Country', 'Competition', 'Season', 'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'VCH', 'VCD', 'VCA', 'WHH', 'WHD', 'WHA']
root
 |-- Country: string (nullable = true)
 |-- Competition: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Div: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- FTR: string (nullable = true)
 |-- B365H: double (nullable = true)
 |-- B365D: double (nullable = true)
 |-- B365A: double (nullable = true)
 |-- BWH: double (nullable = true)
 |-- BWD: double (nullable = true)
 |-- BWA: double (nullable = true)
 |-- IWH: double (nullable = true)
 |-- IWD: double (nullable = true)
 |-- IWA: double (nullable = true)
 |-- LBH: double (nullable = true)
 |-- LBD: double (nullable = true)
 |-- LBA: double (nullable = true)
 |-- VCH: double (nul

In [5]:
df.show(5)

+-------+-------------+---------+---+-------------------+---------+----------------+---+-----+-----+-----+----+---+----+----+---+---+----+---+----+----+----+----+----+----+----+
|Country|  Competition|   Season|Div|               Date| HomeTeam|        AwayTeam|FTR|B365H|B365D|B365A| BWH|BWD| BWA| IWH|IWD|IWA| LBH|LBD| LBA| VCH| VCD| VCA| WHH| WHD| WHA|
+-------+-------------+---------+---+-------------------+---------+----------------+---+-----+-----+-----+----+---+----+----+---+---+----+---+----+----+----+----+----+----+----+
|Belgium|JupilerLeague|2017-2018| B1|2017-07-28 00:00:00|  Antwerp|      Anderlecht|  D| 5.75|  3.8|  1.6| 5.0|4.0|1.67| 4.7|3.6|1.7| 5.2|3.7|1.61| 5.5| 3.9|1.62|4.75|3.75|1.67|
|Belgium|JupilerLeague|2017-2018| B1|2017-07-29 00:00:00|Charleroi|        Kortrijk|  H| 1.62| 3.75|  5.5|1.67|3.7| 5.5| 1.6|3.7|5.4|1.61|3.6| 5.2|1.65| 3.8|5.25|1.65|3.75| 4.8|
|Belgium|JupilerLeague|2017-2018| B1|2017-07-29 00:00:00|    Eupen|         Waregem|  A|  2.6|  3.4|  2.6| 2.7

In [6]:
df.count()

11374

In [7]:
df.describe('Country','Competition','Season','Div','Date','HomeTeam','AwayTeam','FTR').show()

+-------+-------+------------+---------+-----+--------+--------+-----+
|summary|Country| Competition|   Season|  Div|HomeTeam|AwayTeam|  FTR|
+-------+-------+------------+---------+-----+--------+--------+-----+
|  count|  11374|       11374|    11374|11374|   11374|   11374|11374|
|   mean|   null|        null|     null| null|    null|    null| null|
| stddev|   null|        null|     null| null|    null|    null| null|
|    min|Belgium|Championship|2017-2018|   B1|     AEK|     AEK|    A|
|    max| Turkey|      SerieB|2018-2019|   T1|  Zwolle|  Zwolle|    H|
+-------+-------+------------+---------+-----+--------+--------+-----+



In [8]:
# df.describe('WHH','WHD','WHA','SBH','SBD','SBA','IWH','IWD','IWA','GBH','GBD','GBA').show()

In [9]:
dfok = df.dropna(how='any')
print(dfok.count())
dfok.show(5)

7668
+-------+-------------+---------+---+-------------------+---------+----------------+---+-----+-----+-----+----+---+----+----+---+---+----+---+----+----+----+----+----+----+----+
|Country|  Competition|   Season|Div|               Date| HomeTeam|        AwayTeam|FTR|B365H|B365D|B365A| BWH|BWD| BWA| IWH|IWD|IWA| LBH|LBD| LBA| VCH| VCD| VCA| WHH| WHD| WHA|
+-------+-------------+---------+---+-------------------+---------+----------------+---+-----+-----+-----+----+---+----+----+---+---+----+---+----+----+----+----+----+----+----+
|Belgium|JupilerLeague|2017-2018| B1|2017-07-28 00:00:00|  Antwerp|      Anderlecht|  D| 5.75|  3.8|  1.6| 5.0|4.0|1.67| 4.7|3.6|1.7| 5.2|3.7|1.61| 5.5| 3.9|1.62|4.75|3.75|1.67|
|Belgium|JupilerLeague|2017-2018| B1|2017-07-29 00:00:00|Charleroi|        Kortrijk|  H| 1.62| 3.75|  5.5|1.67|3.7| 5.5| 1.6|3.7|5.4|1.61|3.6| 5.2|1.65| 3.8|5.25|1.65|3.75| 4.8|
|Belgium|JupilerLeague|2017-2018| B1|2017-07-29 00:00:00|    Eupen|         Waregem|  A|  2.6|  3.4|  2.6

In [10]:
# Bet Houses selection depending on dataset simulation
print ("Potential bet houses:", bethouses)
betHousesCols = []
betHousesOk = []
for bethouse in bethouses:
    col = bethouse + 'H'
    if col in dfok.columns:
        betHousesOk.append(bethouse)
        betHousesCols.append(bethouse + 'H')
        betHousesCols.append(bethouse + 'D')
        betHousesCols.append(bethouse + 'A')
        
print ("Bet houses selected :" , betHousesOk)
print ("Bet houses columns  :", betHousesCols)

Potential bet houses: ['B365', 'BS', 'BW', 'GB', 'IW', 'LB', 'SB', 'SJ', 'VC', 'WH']
Bet houses selected : ['B365', 'BW', 'IW', 'LB', 'VC', 'WH']
Bet houses columns  : ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA', 'VCH', 'VCD', 'VCA', 'WHH', 'WHD', 'WHA']


In [11]:
# Divergence calculation simulation
match = dfok[betHousesCols].take(1)[0]
print (match)

quotatypes = ['H','D','A']
divergence = 0
for quotatype in quotatypes:
    print ()
    min = 1000
    max = 0
    sum = 0
    count = 0
    for bethouse in betHousesOk:
        col = bethouse + quotatype
        val = match[col]
        if not val is None:
            print (col, ':', val, end='  ')
            sum = sum + val
            count = count + 1
            if val > max:
                max = val
            if val < min:
                min = val
    mean = sum/count
    diverMax = (max / mean) - 1
    if diverMax > divergence:
        divergence = diverMax
    diverMin = abs((min / mean) - 1)
    if diverMin > divergence:
        divergence = diverMin
    
    print ('\nCount:', count, end='  ')
    print ('Sum:', sum, end='  ')
    print ('Max:', max, end='  ')
    print ('Min:', min, end='  ')
    print ('Mean:', mean)
    print ('Diver Max:  ', diverMax, end='  ')
    print ('Diver Min:  ', diverMin, end='  ')
    print ('Divergence: ', divergence)
       

Row(B365H=5.75, B365D=3.8, B365A=1.6, BWH=5.0, BWD=4.0, BWA=1.67, IWH=4.7, IWD=3.6, IWA=1.7, LBH=5.2, LBD=3.7, LBA=1.61, VCH=5.5, VCD=3.9, VCA=1.62, WHH=4.75, WHD=3.75, WHA=1.67)

B365H : 5.75  BWH : 5.0  IWH : 4.7  LBH : 5.2  VCH : 5.5  WHH : 4.75  
Count: 6  Sum: 30.9  Max: 5.75  Min: 4.7  Mean: 5.1499999999999995
Diver Max:   0.11650485436893221  Diver Min:   0.08737864077669888  Divergence:  0.11650485436893221

B365D : 3.8  BWD : 4.0  IWD : 3.6  LBD : 3.7  VCD : 3.9  WHD : 3.75  
Count: 6  Sum: 22.75  Max: 4.0  Min: 3.6  Mean: 3.7916666666666665
Diver Max:   0.05494505494505497  Diver Min:   0.05054945054945048  Divergence:  0.11650485436893221

B365A : 1.6  BWA : 1.67  IWA : 1.7  LBA : 1.61  VCA : 1.62  WHA : 1.67  
Count: 6  Sum: 9.87  Max: 1.7  Min: 1.6  Mean: 1.6449999999999998
Diver Max:   0.033434650455927084  Diver Min:   0.027355623100303816  Divergence:  0.11650485436893221


In [12]:
def filterBetHouses0(df, bethouses):
    betHousesCols = []
    betHousesOk = []
    for bethouse in bethouses:
        col = bethouse + 'H'
        if col in dfok.columns:
            betHousesOk.append(bethouse)
            betHousesCols.append(bethouse + 'H')
            betHousesCols.append(bethouse + 'D')
            betHousesCols.append(bethouse + 'A')

    return (betHousesOk, betHousesCols)

In [13]:
def filterBetHouses(df, bethouses):
    betHousesOk = []
    for bethouse in bethouses:
        col = bethouse + 'H'
        if col in dfok.columns:
            betHousesOk.append(bethouse)
    
    return betHousesOk

In [14]:
def calcDivergence(match, bethouses):
    resultList = ['H','D','A']
    divergence = 0
    for result in resultList:
        min = 1000
        max = 0
        sum = 0
        count = 0
        for bethouse in bethouses:
            col = bethouse + result
            val = match[col]
            if not val is None:
                sum = sum + val
                count = count + 1
                if val > max:
                    max = val
                if val < min:
                    min = val
        mean = sum/count
        diverMax = (max / mean) - 1
        if diverMax > divergence:
            divergence = diverMax
        diverMin = abs((min / mean) - 1)
        if diverMin > divergence:
            divergence = diverMin

    return divergence

In [15]:
# Map divergence
bethousesok = filterBetHouses(dfok, bethouses)
print ("Bet Houses:", bethousesok)

rdddiver = dfok.rdd.map(lambda match: (match, calcDivergence(match, bethousesok)))
#rddlist = dfok.rdd.map(list)
#rdddiver = rddlist.map(lambda match : (match, calcDivergence(match, bethouses)))
display (rdddiver.take(2))

Bet Houses: ['B365', 'BW', 'IW', 'LB', 'VC', 'WH']


[(Row(Country='Belgium', Competition='JupilerLeague', Season='2017-2018', Div='B1', Date=datetime.datetime(2017, 7, 28, 0, 0), HomeTeam='Antwerp', AwayTeam='Anderlecht', FTR='D', B365H=5.75, B365D=3.8, B365A=1.6, BWH=5.0, BWD=4.0, BWA=1.67, IWH=4.7, IWD=3.6, IWA=1.7, LBH=5.2, LBD=3.7, LBA=1.61, VCH=5.5, VCD=3.9, VCA=1.62, WHH=4.75, WHD=3.75, WHA=1.67),
  0.11650485436893221),
 (Row(Country='Belgium', Competition='JupilerLeague', Season='2017-2018', Div='B1', Date=datetime.datetime(2017, 7, 29, 0, 0), HomeTeam='Charleroi', AwayTeam='Kortrijk', FTR='H', B365H=1.62, B365D=3.75, B365A=5.5, BWH=1.67, BWD=3.7, BWA=5.5, IWH=1.6, IWD=3.7, IWA=5.4, LBH=1.61, LBD=3.6, LBA=5.2, VCH=1.65, VCD=3.8, VCA=5.25, WHH=1.65, WHD=3.75, WHA=4.8),
  0.09004739336492884)]

In [16]:
# Top N highest and lower divergence matches of a web - simulation
import datetime

date_ini = '2018-11-01'
date_end = '2018-11-07'
date_ini_obj = datetime.datetime.strptime(date_ini, '%Y-%m-%d')
date_end_obj = datetime.datetime.strptime(date_end, '%Y-%m-%d')
print ("Period: ", date_ini_obj, "to", date_end_obj)

rddperiod = rdddiver.filter(lambda match: match[0][4]>=date_ini_obj).filter(lambda match: match[0][4]<=date_end_obj)
print("Week matches: ", rddperiod.count())

# Top N highest
print ("* Top N highest divergence matches:")
display (rddperiod.top(5, key=lambda match: match[1]))

# Top N lowest
print ("* Top N lowest divergence matches:")
display (rddperiod.top(5, key=lambda match: -match[1]))


Period:  2018-11-01 00:00:00 to 2018-11-07 00:00:00
Week matches:  120
* Top N highest divergence matches:


[(Row(Country='Spain', Competition='Primera', Season='2017-2018', Div='SP1', Date=datetime.datetime(2018, 11, 2, 0, 0), HomeTeam='Barcelona', AwayTeam='Getafe', FTR='D', B365H=1.12, B365D=9.0, B365A=19.0, BWH=1.1, BWD=9.25, BWA=19.0, IWH=1.12, IWD=9.0, IWA=20.0, LBH=1.11, LBD=9.5, LBA=19.0, VCH=1.09, VCD=10.0, VCA=29.0, WHH=1.11, WHD=9.0, WHA=21.0),
  0.3700787401574803),
 (Row(Country='France', Competition='Division2', Season='2017-2018', Div='F2', Date=datetime.datetime(2018, 11, 5, 0, 0), HomeTeam='Le Havre', AwayTeam='Tours', FTR='H', B365H=1.19, B365D=6.0, B365A=15.0, BWH=1.22, BWD=6.0, BWA=10.5, IWH=1.22, IWD=6.25, IWA=10.75, LBH=1.2, LBD=6.0, LBA=13.0, VCH=1.18, VCD=6.5, VCA=17.0, WHH=1.22, WHD=6.0, WHA=9.5),
  0.3465346534653466),
 (Row(Country='Italy', Competition='SerieA', Season='2017-2018', Div='I1', Date=datetime.datetime(2018, 11, 3, 0, 0), HomeTeam='Juventus', AwayTeam='Udinese', FTR='H', B365H=1.22, B365D=6.5, B365A=13.0, BWH=1.25, BWD=5.75, BWA=13.0, IWH=1.25, IWD=6.0,

* Top N lowest divergence matches:


[(Row(Country='Germany', Competition='Liga1', Season='2017-2018', Div='D1', Date=datetime.datetime(2018, 11, 2, 0, 0), HomeTeam='Werder Bremen', AwayTeam='Wolfsburg', FTR='H', B365H=2.14, B365D=3.39, B365A=3.39, BWH=2.1, BWD=3.3, BWA=3.4, IWH=2.1, IWD=3.4, IWA=3.5, LBH=2.05, LBD=3.3, LBA=3.5, VCH=2.15, VCD=3.4, VCA=3.6, WHH=2.05, WHD=3.4, WHA=3.6),
  0.0309671272034302),
 (Row(Country='Germany', Competition='Liga2', Season='2017-2018', Div='D2', Date=datetime.datetime(2018, 11, 2, 0, 0), HomeTeam='Greuther Furth', AwayTeam='Dresden', FTR='H', B365H=2.5, B365D=3.2, B365A=2.79, BWH=2.45, BWD=3.25, BWA=2.85, IWH=2.5, IWD=3.2, IWA=2.75, LBH=2.45, LBD=3.2, LBA=2.75, VCH=2.5, VCD=3.3, VCA=2.88, WHH=2.45, WHD=3.3, WHA=2.7),
  0.03349282296650724),
 (Row(Country='Belgium', Competition='JupilerLeague', Season='2017-2018', Div='B1', Date=datetime.datetime(2018, 11, 3, 0, 0), HomeTeam='Genk', AwayTeam='Gent', FTR='A', B365H=2.1, B365D=3.5, B365A=3.25, BWH=2.2, BWD=3.5, BWA=3.25, IWH=2.1, IWD=3.3,

In [17]:
# Calculate Top N higher and lowest divergence matches in a periode of time

def topNDivergence(rdd, date_ini, date_end):
    
    date_ini_obj = datetime.datetime.strptime(date_ini, '%Y-%m-%d')
    date_end_obj = datetime.datetime.strptime(date_end, '%Y-%m-%d')

    rddperiod = rdd.filter(lambda match: match[0][4]>=date_ini_obj).filter(lambda match: match[0][4]<=date_end_obj)

    rddhigh = rddperiod.top(5, key=lambda match: match[1])
    
    rddlow  = rddperiod.top(5, key=lambda match: -match[1])
    
    count = rdd.count()
    
    return (count, rddhigh, rddlow)


In [18]:
# Example of use of topNDivergence

topNDiver = topNDivergence(rdddiver, '2018-11-01', '2018-11-07')
print ("Matches:", topNDiver[0])
print ("\n* Top N highest divergence matches:")
print (topNDiver[1])
print ("\n* Top N lowest divergence matches:")
print (topNDiver[2])


Matches: 7668

* Top N highest divergence matches:
[(Row(Country='Spain', Competition='Primera', Season='2017-2018', Div='SP1', Date=datetime.datetime(2018, 11, 2, 0, 0), HomeTeam='Barcelona', AwayTeam='Getafe', FTR='D', B365H=1.12, B365D=9.0, B365A=19.0, BWH=1.1, BWD=9.25, BWA=19.0, IWH=1.12, IWD=9.0, IWA=20.0, LBH=1.11, LBD=9.5, LBA=19.0, VCH=1.09, VCD=10.0, VCA=29.0, WHH=1.11, WHD=9.0, WHA=21.0), 0.3700787401574803), (Row(Country='France', Competition='Division2', Season='2017-2018', Div='F2', Date=datetime.datetime(2018, 11, 5, 0, 0), HomeTeam='Le Havre', AwayTeam='Tours', FTR='H', B365H=1.19, B365D=6.0, B365A=15.0, BWH=1.22, BWD=6.0, BWA=10.5, IWH=1.22, IWD=6.25, IWA=10.75, LBH=1.2, LBD=6.0, LBA=13.0, VCH=1.18, VCD=6.5, VCA=17.0, WHH=1.22, WHD=6.0, WHA=9.5), 0.3465346534653466), (Row(Country='Italy', Competition='SerieA', Season='2017-2018', Div='I1', Date=datetime.datetime(2018, 11, 3, 0, 0), HomeTeam='Juventus', AwayTeam='Udinese', FTR='H', B365H=1.22, B365D=6.5, B365A=13.0, BWH

In [19]:
# Obtain Dataframe to make graphic analisys
rddhist = rdddiver.map(lambda match: (match[0][0], match[0][1], match[0][2], match[0][4], match[0][5], match[0][6], round(match[1],2)))
dfhist = rddhist.toDF(['Country','Competition','Season','Date','HomeTeam','AwayTeam','Divergence'])
print ('Matches:', dfhist.count())
print ('Sample data:\n', rddhist.take(2))
dfhist.describe().show()

Matches: 7668
Sample data:
 [('Belgium', 'JupilerLeague', '2017-2018', datetime.datetime(2017, 7, 28, 0, 0), 'Antwerp', 'Anderlecht', 0.12), ('Belgium', 'JupilerLeague', '2017-2018', datetime.datetime(2017, 7, 29, 0, 0), 'Charleroi', 'Kortrijk', 0.09)]
+-------+-------+------------+---------+--------+--------+-------------------+
|summary|Country| Competition|   Season|HomeTeam|AwayTeam|         Divergence|
+-------+-------+------------+---------+--------+--------+-------------------+
|  count|   7668|        7668|     7668|    7668|    7668|               7668|
|   mean|   null|        null|     null|    null|    null|0.08770083463745423|
| stddev|   null|        null|     null|    null|    null|0.05124734175669728|
|    min|Belgium|Championship|2017-2018|     AEK|     AEK|               0.02|
|    max| Turkey|      SerieB|2017-2018|  Zwolle|  Zwolle|               0.57|
+-------+-------+------------+---------+--------+--------+-------------------+



In [20]:
from pyspark_dist_explore import hist
import matplotlib.pyplot as plt

In [24]:
dfhistdiv = dfhist.groupBy('Divergence').count().orderBy(dfhist['Divergence'])
dfhistdiv.show()              

+----------+-----+
|Divergence|count|
+----------+-----+
|      0.02|    7|
|      0.03|  131|
|      0.04|  493|
|      0.05| 1057|
|      0.06| 1299|
|      0.07| 1086|
|      0.08|  773|
|      0.09|  615|
|       0.1|  464|
|      0.11|  320|
|      0.12|  266|
|      0.13|  210|
|      0.14|  171|
|      0.15|  121|
|      0.16|  123|
|      0.17|   83|
|      0.18|   66|
|      0.19|   58|
|       0.2|   49|
|      0.21|   32|
+----------+-----+
only showing top 20 rows



In [26]:
dfhist.groupBy('Country').agg({'Divergence':'mean'}).show()

+-----------+-------------------+
|    Country|    avg(Divergence)|
+-----------+-------------------+
|     Turkey|0.09449826989619377|
|    Germany|0.08013311148086523|
|     France|0.09646825396825388|
|     Greece|0.10354497354497358|
|    Belgium| 0.0829583333333333|
|      Italy| 0.0949939540507859|
|      Spain| 0.0846134969325154|
|    England|0.07769945778466321|
|   Portugal|0.10781456953642368|
|   Scotland|0.09418086500655307|
|Netherlands|0.10542763157894731|
+-----------+-------------------+

